# 0: Setup

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.57.0
!pip install --no-deps trl==0.22.2
!pip install evaluate

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

work_path = '/gdrive/MyDrive/Sungshin/2025-2-NLP/fine-tune-tutorial'

Mounted at /gdrive


### Specify the pretrained model as `LLama-3.1-8B-Instruct-bnb-4bit`

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # context 길이
dtype = None           # GPU에 맞게 자동설정
load_in_4bit = True    # 4bit 양자화

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...",  # 필요하면 HF 토큰
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.9.0+cu126)
    Python  3.12.9 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.9.0+cu126)
    Python  3.12.9 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/

model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

LoRA Setup: Parameter-Efficient Fine-Tuning (PEFT)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.11.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# 1: Data preparation

In [ ]:
from datasets import load_dataset

# Load the dataset from the JSON file
raw_dataset = load_dataset("json", data_files=f"{work_path}/order_analysis-dataset.json", split='train')

Generating train split: 0 examples [00:00, ? examples/s]

##  Q1: Split the dataset into training and test sets using an 80:20 ratio.

In [ ]:
# Q1: 80:20 비율로 train/test 분할
dataset = raw_dataset.train_test_split(test_size=0.2, seed=3407)

# 원본(=train split) 확인
print(dataset["train"].column_names)
print(dataset["train"][0])

['instruction', 'input', 'output']
{'instruction': '너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.', 'input': '주문 문장: 한우스페셜 1인분 주문할게요. 그리고 수제차 메리골드 하나 주세요.', 'output': '- 분석 결과 0: 음식명:한우스페셜,수량:1인분 \n- 분석 결과 1: 음식명:수제차 메리골드, 수량:하나 '}


### Q2: Transform dataset's format int ChatML

instruction -> system

input -> user

output -> assistant

```json
[
    {"role": "system", "content": system},
    {"role": "user", "content": input},
    {"role": "assistant", "content": output}
]
```

In [ ]:
def convert_to_chatml_format(examples):
    systems = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]

    texts = []
    for s, inp, out in zip(systems, inputs, outputs):
        messages = [
            {"role": "system",    "content": s},
            {"role": "user",      "content": inp},
            {"role": "assistant", "content": out},
        ]
        texts.append(messages)

    return {"conversations": texts}

In [ ]:
# train / test 둘 다에 적용
dataset = dataset.map(convert_to_chatml_format, batched=True)

# 변환 결과 예시 확인
dataset["train"][0]

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

{'instruction': '너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.',
 'input': '주문 문장: 한우스페셜 1인분 주문할게요. 그리고 수제차 메리골드 하나 주세요.',
 'output': '- 분석 결과 0: 음식명:한우스페셜,수량:1인분 \n- 분석 결과 1: 음식명:수제차 메리골드, 수량:하나 ',
 'conversations': [{'content': '너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.',
   'role': 'system'},
  {'content': '주문 문장: 한우스페셜 1인분 주문할게요. 그리고 수제차 메리골드 하나 주세요.', 'role': 'user'},
  {'content': '- 분석 결과 0: 음식명:한우스페셜,수량:1인분 \n- 분석 결과 1: 음식명:수제차 메리골드, 수량:하나 ',
   'role': 'assistant'}]}

### Q3: Apply `apply_chat_template`

Roles:
- Special tokens representing system, user, and assistant were added.
- These special tokens are implemented differently across various models, so a unified interface, apply_chat_template, is used.

In [ ]:
from unsloth import apply_chat_template

# conversations → text 로 변환 (모델이 실제로 볼 문자열)
dataset = apply_chat_template(
    dataset,
    tokenizer,
)

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:
dataset["train"][0]

{'instruction': '너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.',
 'input': '주문 문장: 한우스페셜 1인분 주문할게요. 그리고 수제차 메리골드 하나 주세요.',
 'output': '- 분석 결과 0: 음식명:한우스페셜,수량:1인분 \n- 분석 결과 1: 음식명:수제차 메리골드, 수량:하나 ',
 'conversations': [{'content': '너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.',
   'role': 'system'},
  {'content': '주문 문장: 한우스페셜 1인분 주문할게요. 그리고 수제차 메리골드 하나 주세요.', 'role': 'user'},
  {'content': '- 분석 결과 0: 음식명:한우스페셜,수량:1인분 \n- 분석 결과 1: 음식명:수제차 메리골드, 수량:하나 ',
   'role': 'assistant'}],
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n주문 문장: 한우스페셜 1인분 주문할게요. 그리고 수제차 메리골드 하나 주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n- 분석 결과 0: 음식명:한우스페셜,수량:1인분 \n- 분석 결과 1: 음식명:수제차 메리골드, 수량:하나 <|eot_id|>'}

#2: Training the Model
The model is trained using Hugging Face TRL's `SFTTrainer`.

For more detailed information, please refer to the TRL SFT docs.

Due to time constraints, we will only run 60 steps.

When performing actual fine-tuning:
- Set num_train_epochs=1 or more.
- Set max_steps=None.

In [ ]:
from trl import SFTConfig, SFTTrainer

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/2400 [00:00<?, ? examples/s]

In [ ]:
# 학습 전 GPU 메모리 상황 확인
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.883 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,400 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.984100
20,0.767800
30,0.659700
40,0.621000
50,0.579100
60,0.546600


In [ ]:
# 학습 후 메모리/시간 출력
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

381.6844 seconds used for training.
6.36 minutes used for training.
Peak reserved memory = 6.883 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 46.693 %.
Peak reserved memory for training % of max memory = 0.0 %.


# 3: Applying trained model

Apply the trained model to check if it has been successfully fine-tuned.

In [ ]:
system_message = '너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.'

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "짜장면 2그릇, 콜라 1병 주세요."},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 512, pad_token_id = tokenizer.eos_token_id)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


- 분석 결과 0: 음식명:짜장면,수량:2그릇
- 분석 결과 1: 음식명:콜라,수량:1병<|eot_id|>


# 4: Saving and Loading the Fine-Tuned Model
We only save the LoRA adapter (for efficient storage):
- Save to the Hugging Face Hub: `push_to_hub`
- Save locally: `save_pretrained`

In [ ]:
model.save_pretrained(f"{work_path}/lora_model")  # Local saving
tokenizer.save_pretrained(f"{work_path}/lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('/gdrive/MyDrive/Sungshin/2025-2-NLP/fine-tune-tutorial/lora_model/tokenizer_config.json',
 '/gdrive/MyDrive/Sungshin/2025-2-NLP/fine-tune-tutorial/lora_model/special_tokens_map.json',
 '/gdrive/MyDrive/Sungshin/2025-2-NLP/fine-tune-tutorial/lora_model/chat_template.jinja',
 '/gdrive/MyDrive/Sungshin/2025-2-NLP/fine-tune-tutorial/lora_model/tokenizer.json')

# 5: Loading the saved model and tokenizer

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = f"{work_path}/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
pass

==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 256, pad_token_id = tokenizer.eos_token_id)

- 분석 결과 0: 음식명:짜장면, 수량:2그릇
- 분석 결과 1: 음식명:콜라, 수량:1병<|eot_id|>


In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        f"{work_path}/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained(f"{work_path}/lora_model")

# 6: Calculate Metrics

### Q4: Complte the code for calculating BLEU score on the test split

In [ ]:
import evaluate
bleu_metric = evaluate.load("bleu")

In [ ]:
from tqdm import tqdm

test_dataset = dataset['test']

generated_responses = []
reference_responses = []

from transformers import TextStreamer

FastLanguageModel.for_inference(model)

# Text generation loop
for example in tqdm(test_dataset):
    messages = [
        {"role": "system", "content": example["instruction"]},
        {"role": "user",   "content": example["input"]},
    ]

    # 입력 토큰 생성
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True,
        return_tensors = "pt",
    ).to("cuda")

    # 모델로 응답 생성
    outputs = model.generate(
        input_ids,
        max_new_tokens = 128,
        do_sample = False,
        pad_token_id = tokenizer.eos_token_id,
    )

    generated_tokens = outputs[0][input_ids.shape[-1]:]
    generated_text = tokenizer.decode(
        generated_tokens,
        skip_special_tokens = True,
    )

    reference_text = example["output"]

    generated_responses.append(generated_text.strip())
    reference_responses.append([reference_text.strip()])

100%|██████████| 600/600 [39:20<00:00,  3.93s/it]


In [ ]:
result_1 = bleu_metric.compute(
    predictions = generated_responses,
    references = reference_responses,
)

In [ ]:
print(result_1["bleu"])

0.8882550232183097
